# 1. gensim을 이용한 FastText

## 1.1 데이터 다운로드

In [2]:
# 네이버 영화 리뷰 다운
import pandas as pd
import requests

res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')
with open('ratings_train.txt', 'wb') as f:
    f.write(res.content)
    
nsmc = pd.read_csv('ratings_train.txt', sep='\t')    # nsmc에 직접 주소를 입력해서 할 수 도 있지만, 그렇게 되면 매번 직접 연결해야 되기 때문에 용량이 커진다

In [3]:
nsmc

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## 1.2 전처리

In [3]:
import re

# 한글로만 된 글자 찾는 함수
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [4]:
nsmc.loc[0, 'document']

'아 더빙.. 진짜 짜증나네요 목소리'

In [5]:
# 위의 한글만 찾는 함수 이용하여 한글만 찾기
find_hangul(nsmc.loc[0, 'document'])

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [6]:
nsmc['document'].notnull()    # 내용이 있으면 True, 없으면 False

0         True
1         True
2         True
3         True
4         True
          ... 
149995    True
149996    True
149997    True
149998    True
149999    True
Name: document, Length: 150000, dtype: bool

In [7]:
nsmc[nsmc['document'].notnull()]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


### 이 위는 저번에 했던 코드다

In [9]:
# 모든 document가 find_hangul의 영향을 받는다?
data = nsmc[nsmc['document'].notnull()]['document'].map(find_hangul)

In [10]:
data[0]

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [11]:
data[1]

['흠', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나']

In [12]:
def only_hangul(text):
    return ' '.join(find_hangul(text))    # 각 텍스트를 공백을 사이에 두고 합친다?

In [13]:
only_hangul(nsmc.loc[0, 'document'])

'아 더빙 진짜 짜증나네요 목소리'

In [14]:
data2 = nsmc[nsmc['document'].notnull()]['document'].map(only_hangul)

In [15]:
data2[0]

'아 더빙 진짜 짜증나네요 목소리'

In [16]:
with open('nsmc.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(data2))

## 1.3 FastText 모형 학습

In [17]:
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText

In [18]:
# sg = 0(기본값) : cbow학습, sg = 1 : skip-gram
# alpha = 0.025 : 학습률
# min_alpha : 학습률을 낮춰주는 역할
# window = 5 : 좌우 몇개 단어 맥락으로 학습을 할지 결정(2*window)
# min_count = 5 : 최소 몇번 나와야 학습을 함
# vector_size = 100(기본값)

# gensim 4.0 이상의 경우 size=16 대신 vector_size=16 쓸 것
model = FastText(vector_size=16)    # 단어 하나하나 마다 16차원(16개 실수값)으로 임베딩

In [20]:
# gensim 4.0 이상의 경우 sentences=data 대신 corpus_iterable 쓸 것
model.build_vocab(corpus_iterable=data)    # data를 가지고 어휘 학습

# 파일로 불러오기 - 결과는 같다
# model.build_vocab(corpus_file='nsmc.txt')

In [23]:
model.train(
    corpus_iterable = data,
    epochs = 5,
    total_examples = model.corpus_count,
    total_words = model.corpus_total_words
)

(3999437, 5829395)

## 1.4 저장과 불러오기

In [24]:
model.save('nsmc.fastText')

In [25]:
model = FastText.load('nsmc.fastText')

# 2. FastText 임베딩

## 2.1 모형 불러오기

In [27]:
from gensim.models.fasttext import FastText
model = FastText.load('nsmc.fasttext')

## 2.2 단어 임베딩

In [28]:
model.wv.key_to_index

{'영화': 0,
 '너무': 1,
 '정말': 2,
 '진짜': 3,
 '이': 4,
 '그냥': 5,
 '왜': 6,
 '이런': 7,
 '더': 8,
 '점': 9,
 '수': 10,
 '영화를': 11,
 '다': 12,
 '잘': 13,
 '좀': 14,
 '보고': 15,
 'ㅋㅋ': 16,
 '그': 17,
 '영화가': 18,
 '영화는': 19,
 '본': 20,
 '봤는데': 21,
 '최고의': 22,
 '아': 23,
 '이건': 24,
 '내가': 25,
 '드라마': 26,
 '없는': 27,
 '없다': 28,
 '평점': 29,
 '완전': 30,
 '이렇게': 31,
 '참': 32,
 '이거': 33,
 '그리고': 34,
 '이게': 35,
 '좋은': 36,
 '있는': 37,
 '연기': 38,
 '내': 39,
 '평점이': 40,
 '보는': 41,
 '최고': 42,
 '다시': 43,
 '역시': 44,
 '스토리': 45,
 '쓰레기': 46,
 'ㅋ': 47,
 '난': 48,
 '많이': 49,
 '것': 50,
 '한': 51,
 'ㅋㅋㅋ': 52,
 '재밌게': 53,
 '없고': 54,
 '또': 55,
 '하는': 56,
 '아깝다': 57,
 '꼭': 58,
 '보면': 59,
 '마지막': 60,
 '가장': 61,
 '뭐': 62,
 '영화다': 63,
 '무슨': 64,
 '하지만': 65,
 '같은': 66,
 'ㅎㅎ': 67,
 '와': 68,
 '별로': 69,
 '작품': 70,
 '솔직히': 71,
 '끝까지': 72,
 '볼': 73,
 '넘': 74,
 '안': 75,
 '대한': 76,
 '만든': 77,
 '봐도': 78,
 '그래도': 79,
 '시간': 80,
 '같다': 81,
 '전혀': 82,
 '좋다': 83,
 '말이': 84,
 '지금': 85,
 '별': 86,
 '아주': 87,
 '근데': 88,
 '중': 89,
 '뭔가': 90,
 '영화의': 91,
 '하

In [29]:
'히어로' in model.wv.key_to_index    # True면 임베딩이 학습 되어 있다는 의미

True

In [30]:
model.wv['히어로']    # 임베딩 보기

array([-0.44273186,  0.7145135 ,  0.46587405,  0.33758506,  0.9339837 ,
        0.18270324, -0.66692317, -0.5418582 ,  0.1649342 , -0.01837064,
        0.21625474, -0.9388024 ,  0.2714614 , -0.29430625,  0.092889  ,
        0.34704956], dtype=float32)

In [31]:
len(model.wv['히어로'])    # 16차원으로 임베딩한게 적용된걸 볼 수 있다

16

In [32]:
'슈퍼히어로' in model.wv.key_to_index

False

In [33]:
model.wv['슈퍼히어로']    # 학습이 되어 있지 않은데 임베딩은 나온다
# 슈퍼히어로를 글자 단어 앵글로 자른 값과 (슈, 퍼, 히, 어, 로) 기존에 훈련되어 있는
# 애들 중에 같은것들을 더한것

array([-0.17288385,  0.2943131 ,  0.22998524,  0.14934142,  0.3507364 ,
        0.08399435, -0.27074137, -0.22198634,  0.10346054, -0.03544159,
        0.11999439, -0.33406666,  0.05133158, -0.12495674,  0.04693552,
        0.18984325], dtype=float32)

## 2.3 유사도

In [34]:
# 슈퍼히어로와 히어로가 얼마나 비슷한지(1이면 동일, -1이면 전혀 다름)
model.wv.similarity("슈퍼히어로",'히어로')

0.9890265

In [35]:
model.wv.similarity("히어로",'평론가')

0.6938046

In [36]:
# 비슷한 단어 찾아주기
model.wv.most_similar("평론가")

[('점이나', 0.9908576011657715),
 ('점이상은', 0.9905526638031006),
 ('점대지', 0.989378035068512),
 ('점이라', 0.9893098473548889),
 ('점이야', 0.9890960454940796),
 ('점대가', 0.9886109232902527),
 ('점대나', 0.9884304404258728),
 ('점대면', 0.9882683157920837),
 ('점대야', 0.9882405996322632),
 ('점대냐', 0.9882358908653259)]

# 3. FastText를 이용한 감성 분석

## 3.1 준비

In [37]:
from gensim.models.fasttext import FastText
ft = FastText.load('nsmc.fasttext')

In [38]:
import pandas as pd
nsmc = pd.read_csv('ratings_train.txt', sep='\t')

## 3.2 전처리

In [39]:
df = nsmc[nsmc['document'].notnull()]

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
doc_train, doc_test, y_train, y_test = train_test_split(df['document'], df['label'],\
                                                       test_size=0.2, random_state=42)

In [42]:
# 컴퓨터가 알아들을 수 있는 행렬 형태로 변환
import re
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [43]:
import numpy as np
x_train = np.zeros((1000, 16))    # 빈 행렬 생성(1000개의 댓글을 16차원으로 받을 수 있는 행렬)

In [44]:
doc_train.shape

(119996,)

In [45]:
doc_train

31989            아 꿀잼ㅋ 친구랑 봤는데 너무 웃겼음 그리구 김우빈 잘생겼다..
63462                    개건의 졸작 스릴러? 스릴러라고 하게에도 민망하군
17518               장하나 한윤찬 제발 이어주세요 말도안되게 왜 설도현과ㅜㅜㅜ
123410                                   애로영화계의 개OOO
104181                               내용이나 그래픽자체가 허접함
                             ...                    
119882      꿈을 꾸는사람 꿈을 이룬사람 돌멩이도 꿈은 있잖아! 꿈과희망을 주는 영화
103696                                레니 할린.. 이게 뭐니?
131936                                    시라노; 연애조작단
146872                                    집중이 쉽지 않다.
121961    엔딩 장면이 좋고 소소한 일상과 고민과 연애가 공감을 주어 재밌게 보았어여~
Name: document, Length: 119996, dtype: object

In [56]:
#for i, doc in enumerate(doc_train.iloc[:1]):    # 1000개의 데이터 중 1개만 입력
for i, doc in enumerate(doc_train.iloc[:1000]):    # 댓글 1000개 보기
    vs = [ft.wv[word] for word in find_hangul(doc) if word in ft.wv]    # doc에서 한글만 추출해서 word에 보냄. 각 단어의 임베딩 값 리스트로 만듦
    print(doc, vs[0])
    if vs:
        x_train[i,] = np.mean(vs, axis=0)    # 있다면 평균을 만들고 16차원으로 x_train에 넣어줌

아 꿀잼ㅋ 친구랑 봤는데 너무 웃겼음 그리구 김우빈 잘생겼다.. [ 0.5980982   2.3730745   1.5916353  -0.05755538  1.7435817   3.7694666
 -5.0253363  -1.9789917   1.659408   -4.9618716   2.1633682   0.8299153
 -5.5103216  -1.3868638   1.8562533   3.6307776 ]
개건의 졸작 스릴러? 스릴러라고 하게에도 민망하군 [-0.07699517  0.1260558   0.04289925  0.06387204  0.13258812  0.02324208
 -0.05363531 -0.1174444   0.0136532   0.0124048   0.06349776 -0.11165883
  0.04087393 -0.08151393  0.02145614  0.10020899]
장하나 한윤찬 제발 이어주세요 말도안되게 왜 설도현과ㅜㅜㅜ [-0.3338245   0.5360176  -0.03554958  0.30381432  0.48314393  0.15631492
 -0.56833166 -0.38476923 -0.08858851  0.02663341 -0.12775445 -0.44002163
  0.8370189  -0.571526    0.29776478  0.75268704]
애로영화계의 개OOO [-0.09173277  0.30987036  0.1534301   0.18405394  0.21129492  0.02261638
 -0.11585774 -0.23007117  0.01866454 -0.00628038  0.12904876 -0.2553864
  0.13277972 -0.23687428 -0.02724779  0.26680982]
내용이나 그래픽자체가 허접함 [-0.81328243  1.95565    -0.3037667   0.81054634  0.5298371  -0.36078912
  0.2435894  -1.98454

IndexError: list index out of range

In [49]:
# 위의 코드 이해하기 위한 코드
"""
for i, doc in enumerate(doc_train.iloc[:1]):
    for word in find_hangul(doc):
        vs = ft.wv[word]
        print(word, vs)
"""

아 [ 0.5980982   2.3730745   1.5916353  -0.05755538  1.7435817   3.7694666
 -5.0253363  -1.9789917   1.659408   -4.9618716   2.1633682   0.8299153
 -5.5103216  -1.3868638   1.8562533   3.6307776 ]
꿀잼ㅋ [-0.1885376   0.2652854   0.08952474  0.18867624  0.5560314   0.18063216
 -0.40747994 -0.33135143  0.30715176 -0.09261363  0.12688127 -0.15160047
 -0.33995795 -0.10722856  0.16673414  0.31831557]
친구랑 [-1.3032095   0.7762699   0.95079285  0.5171994   1.5533918   0.7985043
 -1.4339343  -0.47293597  1.0024916  -0.11092274  0.6646814  -1.323494
 -0.58860445  0.21454178  0.34748906  0.8520495 ]
봤는데 [-3.916106   -2.4646258   1.4348346   2.9885695   1.1210617   2.61114
 -2.4443371  -1.0839337   2.4493294  -0.76479846 -0.18887213 -1.1381136
 -6.587567   -0.9895793   1.3533307   2.9719708 ]
너무 [-1.1890917   1.9579846  -0.6986587   2.8287308   1.2644832   1.6274803
  0.35607162 -4.45554     0.88652986  3.8561838   0.82784575 -0.00518596
 -4.4444237  -0.6607205   0.45191884  0.93413794]
웃겼음 [-0.30407

In [52]:
x_train

array([[-0.8101936 ,  0.49174821,  0.43802679, ..., -0.42354453,
         0.48609078,  1.1192385 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

## 3.3 모형 학습

In [55]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [57]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
model.fit(x_train, y_train.values[:1000], epochs=1)

32/32 [==============================] - 0s 767us/step - loss: 0.7190 - accuracy: 0.4430


In [59]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 289
Trainable params: 289
Non-trainable params: 0
_________________________________________________________________
